## Autonomous Multi-Agent System for Enhanced Automotive User Experience

#### Capstone Project - Kaggle 5-Day Generative AI Course

In [1]:
!pip uninstall -qqy kfp jupyterlab google-generativeai tensorflow-hub 2>/dev/null
!pip install -U -q langchain langchain-google-genai langgraph google-cloud-aiplatform pandas scikit-learn "langchain-community>=0.2.7" "langchain-core>=0.2.19"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take 

In [2]:
import os
import typing
import uuid
from collections.abc import Iterable
from random import randint
from pprint import pprint

from langchain_core.messages.ai import AIMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from google.cloud import aiplatform
from langchain_community.embeddings import VertexAIEmbeddings
from langchain_community.vectorstores.matching_engine import MatchingEngine

from IPython.display import Image, display, Markdown

from typing import Annotated, Literal
from typing_extensions import TypedDict

In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
PROJECT_ID = "capstone-project"
LOCATION = "us-central1"

try:
    aiplatform.init(project=PROJECT_ID, location=LOCATION)
    print(f"Vertex AI initialized for project {PROJECT_ID} in {LOCATION}.")
except Exception as e:
    print(f"Error initializing Vertex AI: {e}")
    print("Please ensure you have entered a valid PROJECT_ID and have the necessary permissions.")


Vertex AI initialized for project capstone-project in us-central1.


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    current_request: str | None
    last_agent_response: str | None
    route_decision: Literal["Navigation", "Media", "Communication", "VehicleInfo", "Respond", "Error", None]
    vehicle_context: list[str] | None
    navigation_context: dict | None
    media_context: dict | None
    communication_context: dict | None
    error: str | None
    final_response_generated: bool